in another notebook, do market analysis:
- how much average income for airbnb in austin
- what is the best time for airbnb
- what regions are the best for higher income, in those regions, what is the earning for cases with different number of rooms or beds. - How would you leverage house price vs. monthly income, find the best regions (close to the center or far, show on map return be dollor investment)
- how the number of rooms and bath affect income
- what factors like experience or review could do for us
- what factors should be considered to maximize the income 
- for each zipcode, we will plot income for factors like number of beds and baths, and will show average income.